<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

В качестве признаков для моделирования получена информация о поле, возрасте, зарплате застрахованного, количестве членов его семьи.
Целевым признаком является количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/datasets/insurance.csv')

display(df.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


С точки зрения исходных данных нареканий нет. Пропуски отсутствуют, нереалистичных выбросов также не наблюдается.

## Умножение матриц

<b>Утверждение</b>: При умножении матрицы признаков на обратимую матрицу качество линейной регрессии не изменится, так как не изменятся её параметры.

<b>Доказательство: </b> <br>
Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Умножим $X$ на $P$, введем вектор $w_p = P^{-1} w$, подставим в исходную формулу:
$$
P^{-1} w = ((XP)^T XP)^{-1} (XP)^T y
$$

Преобразуем формулу:

$$
((XP)^T X P)^{-1} (XP)^T y = (P^T X^T X P)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T = P^{-1} (X^T X)^{-1} I X^T
$$
Итого:
$$
P^{-1} w = P^{-1} (X^T X)^{-1} I X^T,
$$
что указывает на отсутствие различий при перемножении $X$ на матрицу $P$

## Алгоритм преобразования

**Алгоритм**

Оптимальным методом кажется генерация случайной матрицы и умножение её на матрицу фичей. 
1. Сгенерируем рандомную матрицу размером $k*k$, где $k$ - количество признаков;
2. Матрицу признаков умножаем на получившуюся рандомную матрицу;
3. Построим модель с применением преобразованной матрицы;
4. Определим метрики модели.

**Обоснование**
Качество линейной регрессии не изменится, так как будет произведено умножение матрицы фичей на обратимую матрицу, доказательство отсутствия изменений метрик модели при изменении матрицы таким образом приведено в шаге 2.

В качестве альтернативы возможно создание кастомной матрицы из матрицы фичей путем перестановки столбцов/строк, умножение её на число, но криптографической сложности это не добавит. Размотать клубок того, какая матрица была сгенерирована рандомайзером и умножена на матрицу фичей, гораздо сложнее.

## Проверка алгоритма

Сформируем матрицу, проверим её обратимость:

In [5]:
target = df['Страховые выплаты']
features = df.drop(['Страховые выплаты'], axis = 1)
seed_no = 42
try:
    np.random.seed(seed = seed_no)
    matrix_rand = np.random.normal(size = (4,4))
    matrix_rand_inv = np.linalg.inv(matrix_rand)
except numpy.linalg.LinAlgError:
    print('Матрица не обратима, измените условия генерации.')
print('Случайно сгенерированная матрица: \n', matrix_rand)

Случайно сгенерированная матрица: 
 [[ 0.49671415 -0.1382643   0.64768854  1.52302986]
 [-0.23415337 -0.23413696  1.57921282  0.76743473]
 [-0.46947439  0.54256004 -0.46341769 -0.46572975]
 [ 0.24196227 -1.91328024 -1.72491783 -0.56228753]]


In [6]:
features_randomized = features @ matrix_rand
print('Общий вид преобразованных фичей: \n', features_randomized.head())
model = LinearRegression()
model.fit(features, target)
prediction_real = model.predict(features)
print()
print('r2 линейной регрессии на исходных данных - ', r2_score(target , prediction_real))
model.fit(features_randomized, target)
prediction_randomized = model.predict(features_randomized)
print()
print('r2 линейной регрессии с преобразованными фичами - ', r2_score(target , prediction_randomized))

Общий вид преобразованных фичей: 
               0             1             2             3
0 -23294.791154  26899.327002 -22921.847067 -23067.770211
1 -17850.555758  20604.598076 -17538.953455 -17662.990926
2  -9865.752553  11386.970944  -9685.974377  -9758.069218
3 -19581.515190  22616.010381 -19294.804157 -19405.939170
4 -12259.341053  14154.123038 -12050.336135 -12132.535366

r2 линейной регрессии на исходных данных -  0.42494550286668

r2 линейной регрессии с преобразованными фичами -  0.424945502866682


Качество модели не изменилось после преобразования данных. Для вариативности seed рандомайзера можно привязать к текущей дате.